In [10]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

dir_DORA = Path('DORA')

In [9]:
v_model = 'KA51AG8742'

file = 'dora_meta.csv'
meta = pd.read_csv(dir_DORA/file , index_col=0)
meta['timestamp_conv'] = meta['timestamp_conv'].astype('datetime64[ms]')

meta_vehicle_id  = (meta.groupby(['vehicle_id', 'event_id']).id.count()
                    .reset_index()
                    # .drop(['id'], axis=1)
                    )
meta_vehicle_id 

,vehicle_id,event_id,id
0,KA01AF2992,1,10
1,KA01AF2992,2,14
2,KA01AF2992,3,10
3,KA01AF2992,4,10
4,KA01AF2992,7,10
...,...,...,...
7967,MH04KF8187,1093,10
7968,MH04KF8187,1094,10
7969,MH04KF8187,1095,10
7970,MH04KF8187,1096,10


### load label

In [19]:

col_sel = ['event_id', 'anomaly']
label = pd.read_csv(dir_DORA/'result_AbnormalDetect.csv', index_col=0)[col_sel]
label['anomaly'] = label.anomaly.astype('uint8')
label


,event_id,anomaly
0,1,1
1,2,1
2,3,0
3,4,1
4,5,1
...,...,...
3300,3301,0
3301,3302,0
3302,3303,1
3303,3304,0


### split train and test by event_id

In [28]:
file_ = 'dora'
# np.savez_compressed(save_path/f'{file_}.npz', sensor=data_array)
data_array = np.load(dir_DORA/f'{file_}.npz',)['sensor']
print(data_array.shape)

(7972, 17, 6)


In [50]:

X_train, X_test, y_train, y_test = train_test_split(label.event_id.values, label.anomaly.values, test_size=0.3, random_state=42, stratify=label.anomaly.values)
print(len(X_train), len(X_test))

2313 992


In [51]:
for idx_ ,file_ in zip([X_train, X_test], ['train', 'test']):
    np.savez_compressed(dir_DORA/f'X_{file_}.npz', sensor=data_array[idx_])


In [52]:
for idx_ , file_ in zip([y_train, y_test], ['train', 'test']):
    np.save(dir_DORA/f'y_{file_}.npy', idx_) 